# Part1. Prepare datasets
*   Merge data from Yelp with Google places database



In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
from datetime import timedelta
import copy
import requests
import json
import time
from google.colab import files
import urllib.parse
import csv
import re

In [0]:
clean_rest_name = "Popeye's SupplÃ©ments MontrÃ©al"
# clean_rest_name = re.sub(r'[^a-zA-Z0-9_\s&\']+', '', raw_rest_name2)
url_rest_name = urllib.parse.quote(clean_rest_name)
url_rest_name

## Load data from csv

In [0]:
################################################################################
######################### Data loader ###########################################
################################################################################
#Data loader
raw_data_array = list(csv.reader(open("/content/drive/My Drive/DataFest/merged_rest_info_new.csv")))
raw_data_array.pop(0)
raw_data = pd.read_csv("/content/drive/My Drive/DataFest/merged_rest_info_new.csv",encoding = "ISO-8859-1")
clean_data = copy.deepcopy(raw_data)
lat_lst = list(raw_data.latitude)
lng_lst = list(raw_data.longitude)
rest_name_lst = list(raw_data.name)



In [0]:
len(rest_name_lst)

## Determine the closure of each restaurant using Google places API

In [0]:
#Parameters
num_closed_temp = 0
# num_closed_perm = 0
num_inv_coord = 0
num_rest_processed = 0

for idx in range(42002, len(rest_name_lst)):
  num_rest_processed += 1
  print("num_rest_processed:{}  ".format(num_rest_processed))

  #Parsing the name of Restaurant
  raw_rest_name = raw_data_array[idx][3]
  clean_rest_name = re.sub(r'[^a-zA-Z0-9_\s&\']+', '', raw_rest_name)
  url_rest_name = urllib.parse.quote(clean_rest_name)
  print("\traw_rest_name:{}  clean_rest_name:{} ".format(raw_rest_name, clean_rest_name))

  #Parsing latitude and longitude
  lat_y, lng_y = lat_lst[idx], lng_lst[idx]

  #Fetch business status
  url_prefix = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
  url_input = "input=" + url_rest_name
  url_inputtype = "&inputtype=textquery"
  url_fiels = "&fields=business_status,formatted_address,name,rating,place_id,geometry"
  url_locationbias = "&locationbias=point:" + str(lat_y) + "," + str(lng_y)
  url_key = "&key=" + api_key
  url = url_prefix + url_input + url_inputtype + url_fiels + url_locationbias + url_key
  respon = requests.get(url)
  results = json.loads(respon.text)['candidates']

  # If google api find resturant information
  if len(results) != 0 and "business_status" in results[0]:
    result = results[0]
    status = result["business_status"]
    lat_g, lng_g = result['geometry']['location']['lat'], result['geometry']['location']['lng']
    # Validate coordinates
    if abs(lat_y - float(lat_g)) < 0.05 and abs(lng_y - float(lng_g)) < 0.05:
      # Update clean data frame
      clean_data.loc[idx,"Found_by_g"] = True
      clean_data.loc[idx,"status_g"] = status
      clean_data.loc[idx,"name_g"] = result['name']
      clean_data.loc[idx,"place_id_g"] = result['place_id']
      clean_data.loc[idx,"lat_g"] = lat_g
      clean_data.loc[idx,"lng_g"] = lng_g
      if "rating" in result:
        clean_data.loc[idx,"rating_g"] = result['rating']
      # clean_data.loc[idx,"types_g"] = result['types']
      clean_data.loc[idx,"formatted_address_g"] = result['formatted_address']
      
      print("\tstatus:{}   ".format(status))
      if status == "CLOSED_TEMPORARILY":
          num_closed_temp += 1
          print("num_closed_temp:{}".format(num_closed_temp))
    else:
      print("\tNot found:Invalid coordinates")
      clean_data.loc[idx,"More"] = "CoordError"
      clean_data.loc[idx,"Found_by_g"] = False
      num_inv_coord += 1
      print("num_inv_coord:{}".format(num_inv_coord))
  else:
    print("\tNot found:No resturant information")
    clean_data.loc[idx,"More"] = "NoInfo"
    clean_data.loc[idx,"Found_by_g"] = False

clean_data.to_csv('/content/drive/My Drive/DataFest/merged_rest_info_new.csv')

In [0]:
clean_data.to_csv('/content/drive/My Drive/DataFest/upto_34901_merged_rest_info.csv')

## Merge datasets 

# Part2. Preprocess data





## Helper code




In [0]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset, DataLoader

In [0]:
raw_data = pd.read_csv("/content/drive/My Drive/DataFest/merged_rest_info.csv")

In [0]:
raw_data

In [0]:
rest_info_sequence = raw_data.iloc[[0, 3], [3, 4]]
rest_info = rest_info_sequence.to_numpy(dtype='float') ## Using int?
rest_info


In [0]:
normed_matrix = normalize(rest_info, axis=0, norm='l1')
normed_matrix

In [0]:
dummy = np.array([[-1 , 0, 1], [-1,0,  1]])
normed_matrix = normalize(dummy, axis=0, norm='l1')
normed_matrix

In [0]:
pd.get_dummies([1 , 2 ,3]).to_numpy()

## Random Split

In [0]:
class DatasetFromCSV(Dataset):
    def __init__(self, csv_path, transform=None):
        self.data = pd.read_csv(csv_path)
        # self.labels = pd.get_dummies(self.data['labels']).to_numpy()
        self.labels = self.data['labels']
        # self.labels = self.labels.replace({1:0})
        # self.labels = self.labels.replace({2:1})
        self.count_row  = self.data.shape[0] 
        self.count_col  = self.data.shape[1]
        # self.rests_info = self.data.iloc[:,[0,self.count_col]].to_numpy(dtype='float') ## Using int?  
        self.rests_info = self.data.drop('labels', axis=1).to_numpy(dtype='float32')

    def __getitem__(self, index):
        # This method should return only 1 sample and label 
        # (according to "index"), not the whole dataset
        rest_info = self.rests_info[index]
        label = self.labels[index]
        return rest_info, label

    def __len__(self):
        return len(self.labels)

In [0]:
data = pd.read_csv("/content/drive/My Drive/DataFest/clean_binary_6340_7000.csv")

In [0]:
data.shape

In [0]:
rests_info = data.drop('labels', axis=1).to_numpy(dtype='float32')
rests_info.shape

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(rests_info)

In [0]:
pca.singular_values_

In [0]:
import numpy as np
from sklearn.manifold import TSNE
data_embedded = TSNE(n_components=2).fit_transform(rests_info)
data_embedded.shape

In [0]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 2)
sns.scatterplot(data_embedded[:,0], data_embedded[:,1], hue=data['labels'], legend='full', palette=palette)

In [0]:
labels = data['labels']

In [0]:
labels = pd.get_dummies(data['labels']).to_numpy()

In [0]:
np.shape(labels)

In [19]:
labels.value_counts()

2    3659
0    3382
1    2958
Name: labels, dtype: int64

In [0]:
rests_info = data.drop('labels', axis=1).to_numpy(dtype='float')

In [0]:
np.shape(rests_info)

(10340, 108)

In [0]:
# dataset = DatasetFromCSV("/content/drive/My Drive/DataFest/clean_binary_6340_7000.csv")
dataset = DatasetFromCSV("/content/drive/My Drive/DataFest/6.12/clean_multi_6340_7000.csv")
batch_size = 100
validation_split = .2
shuffle_dataset = True
random_seed= 20

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [0]:
train_indices

In [99]:
dataset.labels.value_counts()

2    7000
0    3382
1    2958
Name: labels, dtype: int64

#Part3. Main Loop

In [0]:
# import torch.nn as nn
# import torch.nn.functional as F

# inputs, n_hidden0, n_hidden1, out = 91, 64, 28, 3
# model = nn.Sequential(
#     nn.Linear(inputs, n_hidden0, bias = True), 
#     nn.relu(), 
#     nn.Linear(n_hidden0, n_hidden1, bias = True),
#     nn.relu(),
#     nn.Linear(n_hidden1, out, bias = True),
#     # nn.Softmax()
# )
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

## Design model


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, inputs, n_hidden0, n_hidden1, out):
        super(Net, self).__init__()
        self.n_hidden0 = n_hidden0
        self.n_hidden1 = n_hidden1
        self.fc1 = nn.Linear(inputs, n_hidden0)
        self.fc2 = nn.Linear(n_hidden0, n_hidden1)
        self.fc3 = nn.Linear(n_hidden1, out)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class NetWithDropOut(nn.Module):
    def __init__(self, inputs, n_hidden0, n_hidden1, out):
        super(NetWithDropOut, self).__init__()
        self.n_hidden0 = n_hidden0
        self.n_hidden1 = n_hidden1
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(inputs, n_hidden0)
        self.fc2 = nn.Linear(n_hidden0, n_hidden1)
        self.fc3 = nn.Linear(n_hidden1, out)

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        # x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        # x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=4, stride=1, padding=0)
        self.pool = nn.MaxPool1d(3, 2)
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=2, stride=1, padding=0)
        self.fc1 = nn.Linear(25, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = CNN()

In [0]:
# pool of size=3, stride=2
m = nn.MaxPool1d(3, stride=2)
input = torch.randn(1, 1, 109)
# >>> output = m(input)

In [0]:
conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=4, stride=1, padding=0)
output = conv1(input)
pool = nn.MaxPool1d(3, 2)
output = pool(output)
conv2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=2, stride=1, padding=0)
output = pool(conv2(output))

In [0]:
input = torch.randn(30, 109)
output = input.unsqueeze(1).unsqueeze(1)

In [0]:
output.size()

torch.Size([1, 1, 25])

In [0]:
output.size()

torch.Size([1, 1, 54])

In [0]:
class MyNetBN(nn.Module):
    def __init__(self): 
        super(MyNetBN, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(109, 48),
            nn.BatchNorm1d(48), #applying batch norm
            nn.ReLU(),
            nn.Linear(48, 24),
            nn.BatchNorm1d(24),
            nn.ReLU(),
            nn.Linear(24, 2)
        )
             
    def forward(self, x):
        x = self.classifier(x)
        return x

model = MyNetBN()

In [0]:
class MyNetBN(nn.Module):
    def __init__(self,n_hidden0,n_hidden1): 
        super(MyNetBN, self).__init__()
        self.n_hidden0 = n_hidden0
        self.n_hidden1 = n_hidden1
        self.classifier = nn.Sequential(
            nn.Linear(109, 34),
            nn.Dropout(0.2), #50 % probability 
            nn.ReLU(),
            nn.Linear(34, 20),
            nn.Dropout(0.2), #50 % probability 
            nn.ReLU(),
            nn.Linear(20, 2),
        )
             
    def forward(self, x):
        x = self.classifier(x)
        return x

##Construct loss and optimizer

In [0]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

##Train and Test

In [0]:
def train(epoch,criterion,optimizer):
# def train(epoch):
  average_loss = 0
  length = 0
  for batch_idx, (inputs, target) in enumerate(train_loader):
      length += 1
      optimizer.zero_grad()

      #forward + backward + update  
      # outputs = model(inputs).squeeze()
      outputs = model(inputs)
      loss = criterion(outputs, target)
      loss.backward()
      optimizer.step()

      # print('epoch: ', epoch + 1, 'batch_idx: ', batch_idx + 1 ,' loss: ', loss.item())
      # if (batch_idx % 100 == 0):
      #   print('[%d, %5d] loss: %f' % (epoch + 1, batch_idx + 1, loss.item()))
      average_loss += loss.item()
  print('[%d] loss: %f' % (epoch + 1, average_loss/length))
  return average_loss/length

In [0]:
def test():
    correct = 0
    total = 0
    # label_num = 2
    label_num = 3
    class_correct = list(0. for i in range(label_num))
    class_total = list(0. for i in range(label_num))
    class_accuracy = list(0. for i in range(label_num))
    average_accuracy = 0
    # all_label = [0,1]
    all_label = [0, 1, 2]
    model.eval()
    with torch.no_grad():
        for data in validation_loader:
            restaurants, labels = data
            # outputs = model(restaurants).squeeze()
            outputs = model(restaurants)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    print('Total Accuracy of the network on the %d test restaurant: %d %%' % (total, 100 * correct / total))
    for i in range(label_num):
        print('Accuracy of %5s : %2d %%' % (
            all_label[i], 100 * class_correct[i] / class_total[i]))
        class_accuracy[i] = class_correct[i] / class_total[i]
    average_accuracy = sum(class_accuracy)/label_num
    print('Average Accuracy of the network on the %d test restaurant: %d %%' % (total, 100 * average_accuracy))
    return 100 * correct / total

In [0]:
def test_with_class_output():    
    class_correct = list(0. for i in range(3))
    class_total = list(0. for i in range(3))
    with torch.no_grad():
        for data in validation_loader:
            restaurants, labels = data
            outputs = model(restaurants)
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == labels).squeeze()
            for i in range(3):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(3):
        print('Accuracy of %5s : %2d %%' % (
            label[i], 100 * class_correct[i] / class_total[i]))

In [0]:
#####--Selection--loop---###
highest_accuracy = 0
h0 = 0
h1 = 0
Epochs = []
Loss = []
valaccuracy = []
for hidden_0 in [32, 48, 64, 128]:
    for hidden_1 in [14, 20, 32, 48, 60]:
        model = NetWithDropOut(109, hidden_0, hidden_1, 3)
        # model = MyNetBN(hidden_0, hidden_1)
        print('num_hidden 0 :%d; num_hidden 1: %d' % (model.n_hidden0, model.n_hidden1))
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.2)
        for epoch in range(1500):
            Epochs.append(epoch)
            loss = train(epoch,criterion,optimizer)
            Loss.append(loss)
            accuracy = test()
            valaccuracy.append(accuracy)
            if highest_accuracy < accuracy: 
                highest_accuracy = accuracy
                h1=hidden_1
                h0=hidden_0

In [0]:
highest_accuracy

In [0]:
valaccuracy

In [0]:
h0

In [0]:
h1

In [0]:
# num_epochs = 10
# for epoch in range(num_epochs):
#     # Train:   
#     for batch_index, (data, labels) in enumerate(train_loader):
#         pred = model(data)
#         loss = criterion(pred, labels)
#         print('epoch: ', epoch + 1, 'batch_index: ', batch_index + 1 ,' loss: ', loss.item())
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

In [0]:
import csv
import numpy as np
valaccuracy = np.array(valaccuracy)
Loss = np.array(Loss)
Epochs = np.array(Epochs)

with open('/content/drive/My Drive/DataFest/6.12/learning_curve_multi.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in range(0,Epochs.shape[0]):
        finalList = [Epochs[row],valaccuracy[row],Loss[row]]
        writer.writerow(finalList)

# Part4. Predict result

In [0]:
dataiter = iter(validation_loader)
restaurants, labels = dataiter.next()
classes = [0, 1]

# print restaurants
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

outputs = model(restaurants)
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

GroundTruth:      1     0     0     1
Predicted:      1     1     1     1


#Part5. Save and load model


In [0]:
torch.save(model, PATH)

In [0]:
# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

#K-Fold Cross Validation

In [0]:
!pip install -U skorch
from skorch import NeuralNetClassifier
from sklearn.model_selection import cross_val_score

In [0]:
# import data
df = pd.read_csv("/content/drive/My Drive/DataFest/clean_binary_6340_7000.csv")
df = df.sample(frac=1)

X_data = df[df.columns[:-1]].values.astype(np.float32)
y_data = df["labels"].astype("category").cat.codes.values.astype(np.int64)
print(X_data.shape, y_data.shape)

In [0]:
# generate skorch high-level classifier and perform 5-fold cross validation using cross_val_score()
logistic = NeuralNetClassifier(model, max_epochs = 10, lr = 1e-2)
scores = cross_val_score(logistic, X_data, y_data, cv = 5, scoring = "accuracy")

In [0]:
# print out results
print(scores)
print(scores.mean(), scores.std())

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/My Drive/DataFest/clean_binary_6340_7000.csv")
df = df.sample(frac=1)
X_train = df[df.columns[:-1]].values.astype(np.float32)
y_train = df["labels"].astype("category").cat.codes.values.astype(np.int64)
clf = MLPClassifier((128,128),random_state=1, max_iter=3000).fit(X_train[:11000], y_train[:11000])

In [0]:
X_train = df[df.columns[:-1]].values.astype(np.float32)

In [0]:
X_train[:11000]

In [0]:
clf.score(X_train[11001:], y_train[11001:])